# 04 Method 1 - Linear SVM Classifier

In [2]:
import numpy as np #Handle computations 
import matplotlib.pyplot as plt #Visualisations
import pandas as pd #I/O

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, normalize, scale
from sklearn.svm import SVC #Support Vector Machine Classification
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix


In [ ]:
#Import the Dataset

text_vector = pd.read_csv('')

X = text_vector[:-1]
y = text_vector[-1]

In [ ]:
#scaler
# X_scaled = scale(X)
#normalize
# X_normalized = normalize(X)

In [ ]:
# Training 50%, Testing 50%
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.5, random_state=42)

In [ ]:
# Conduct SVM Classification
svm_clf = SVC(probability=True, random_state=42)
svm_clf.fit(X_train, y_train)

y_score = svm_clf.predict_proba(X_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_score)

In [ ]:
plt.figure(figsize=(8, 6))
plt.title('ROC Curve')
plt.plot(fpr, tpr, "g--", linewidth=2)
plt.plot([0, 1], [0, 1], 'k--')
plt.axis([0, 1, 0, 1])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
y_pred = model.predict(X_test)
#summarize the fit
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))